In [2]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv(r"C:\Users\ZHUO\Desktop\dataset.csv",index_col=0)
data.head()

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode
0,0,132.0,71.0,153,153,710.0,153,193.4,0.00062,5.5764,...,15.0,15.0,15.658145,41.966027,-87.646939,178.671811,145.828347,13.563011,4.83519,15.0
1,50,156.0,82.0,1318,1318,1318.0,1115,178.6,0.55216,0.0000,...,15.0,15.0,15.658145,41.966027,-87.646939,178.671811,145.828347,13.563011,4.83519,15.0
2,0,156.0,64.0,152,152,710.0,152,181.6,0.00036,3.1572,...,15.0,15.0,15.658145,41.975176,-87.650256,192.049602,155.442131,13.040247,5.98351,15.0
3,12,166.0,116.0,1643,1643,1643.0,1479,209.8,0.50847,0.1692,...,15.0,15.0,15.658145,41.975176,-87.650256,192.049602,155.442131,13.040247,5.98351,15.0
4,0,138.0,72.0,425,425,710.0,425,229.4,0.00000,2.9232,...,15.0,15.0,15.658145,41.964429,-87.642942,178.924319,156.492138,13.428711,14.54317,15.0


In [4]:
data['heart_rate_change_value']=data['heart_rate_stop']-data['heart_rate_start']
data=data.drop(['altitude_change_past_7200_to_10800_seconds','altitude_change_past_10800_to_14400_seconds'],axis=1)
#data.head() #(4547,184)
data.shape
data.head()

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
0,0,132.0,71.0,153,153,710.0,153,193.4,0.00062,5.5764,...,15.0,15.658145,41.966027,-87.646939,178.671811,145.828347,13.563011,4.83519,15.0,61.0
1,50,156.0,82.0,1318,1318,1318.0,1115,178.6,0.55216,0.0000,...,15.0,15.658145,41.966027,-87.646939,178.671811,145.828347,13.563011,4.83519,15.0,74.0
2,0,156.0,64.0,152,152,710.0,152,181.6,0.00036,3.1572,...,15.0,15.658145,41.975176,-87.650256,192.049602,155.442131,13.040247,5.98351,15.0,92.0
3,12,166.0,116.0,1643,1643,1643.0,1479,209.8,0.50847,0.1692,...,15.0,15.658145,41.975176,-87.650256,192.049602,155.442131,13.040247,5.98351,15.0,50.0
4,0,138.0,72.0,425,425,710.0,425,229.4,0.00000,2.9232,...,15.0,15.658145,41.964429,-87.642942,178.924319,156.492138,13.428711,14.54317,15.0,66.0


In [10]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression as MIR
import warnings
warnings.filterwarnings('ignore')

X=data.iloc[:,:-1]#(4547,183)
y=data.iloc[:,-1]

#方差过滤  4547,183 未剔除
X_fsvar=VarianceThreshold().fit_transform(X)
X_fsvar.shape


#相关性过滤 互信息 MIR  4547,176
result=MIR(X_fsvar,y)
k=result.shape[0]-sum(result<=0)
selector=SelectKBest(MIR,k)
X_fsMIR=selector.fit_transform(X_fsvar,y)
#X_fsMIR.shape


mask = selector.get_support()
new_features = []
for bool, feature in zip(mask, X.columns):
    if bool:
        new_features.append(feature)
        
dataframe=pd.DataFrame(X_fsMIR,columns=new_features)
#dataframe.shape #4547,177
dataframe.head()
#dataframe.to_csv('datacv.csv')

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode
0,0.0,132.0,71.0,153.0,153.0,710.0,153.0,193.4,0.00062,5.5764,...,15.0,15.0,15.658145,41.966027,-87.646939,178.671811,145.828347,13.563011,4.83519,15.0
1,50.0,156.0,82.0,1318.0,1318.0,1318.0,1115.0,178.6,0.55216,0.0000,...,15.0,15.0,15.658145,41.966027,-87.646939,178.671811,145.828347,13.563011,4.83519,15.0
2,0.0,156.0,64.0,152.0,152.0,710.0,152.0,181.6,0.00036,3.1572,...,15.0,15.0,15.658145,41.975176,-87.650256,192.049602,155.442131,13.040247,5.98351,15.0
3,12.0,166.0,116.0,1643.0,1643.0,1643.0,1479.0,209.8,0.50847,0.1692,...,15.0,15.0,15.658145,41.975176,-87.650256,192.049602,155.442131,13.040247,5.98351,15.0
4,0.0,138.0,72.0,425.0,425.0,710.0,425.0,229.4,0.00000,2.9232,...,15.0,15.0,15.658145,41.964429,-87.642942,178.924319,156.492138,13.428711,14.54317,15.0


In [17]:
#归一化
from sklearn.preprocessing import MinMaxScaler
#x=data.drop(columns=['heart_rate_change_value'],axis=1).values
scaler = MinMaxScaler()  #实例化
X= scaler.fit_transform(dataframe)
y=scaler.fit_transform(y.values.reshape(-1,1))

In [18]:
#数据集分离
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=10)

In [25]:
#将X_train和y_train合并
xt=pd.DataFrame(X_train)
yt=pd.DataFrame(y_train)
xt.columns=dataframe.columns
yt.columns=['heart_rate_change_value']


xyt=pd.concat((xt,yt),axis=1)
xyt.head()
#xyt.shape 3182,177

xyt.to_csv('train_fsMIR_scaler.csv')  #3182

In [26]:
#将X_test和y_test合并
xe=pd.DataFrame(X_test)
ye=pd.DataFrame(y_test)
xe.columns=dataframe.columns
ye.columns=['heart_rate_change_value']
xye=pd.concat((xe,ye),axis=1)
xye.head()
#xye.shape  1365,177

xye.to_csv('test_fsMIR_scaler.csv')

In [28]:
t=data.iloc[:,-1]
t.head()

0    61.0
1    74.0
2    92.0
3    50.0
4    66.0
Name: heart_rate_change_value, dtype: float64

In [29]:
#未经归一化后的数据集分离
from sklearn.model_selection import train_test_split
X_train1,X_test1,y_train1,y_test1=train_test_split(dataframe,t,test_size=0.3,random_state=10)

In [52]:
#将X_train1和y_train1合并
xt1=pd.DataFrame(X_train1)
yt1=pd.DataFrame(y_train1)
xt1.columns=dataframe.columns
yt1.columns=['heart_rate_change_value']

xyt1=pd.concat((xt1,yt1),axis=1)
xyt1.head()
#xyt1.shape #3182,177

#xyt1.to_csv('train_fsMIR.csv')

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
1404,0.0,161.0,71.0,164.0,164.0,710.0,164.0,178.2,0.00000,6.0480,...,23.0,23.510820,41.951512,-87.643213,174.734274,143.381054,12.605056,13.31553,23.0,90.0
3636,90.0,96.0,134.0,836.0,813.0,352.0,48.0,157.0,2.32925,10.4472,...,30.0,30.193087,41.963483,-87.647441,175.247647,154.753239,12.247823,5.90091,31.0,-38.0
1561,11.0,162.0,135.0,1431.0,1389.0,1200.0,1035.0,198.8,1.38810,4.4676,...,10.0,11.328488,41.965081,-87.647290,179.409400,153.560481,14.031540,5.47555,10.0,27.0
778,3.0,137.0,145.0,2234.0,2144.0,1475.0,317.0,161.4,6.62182,10.1448,...,13.0,13.290435,41.958440,-87.645287,162.821807,148.373588,12.732593,8.22650,13.0,-8.0
4328,54.0,170.0,81.0,1553.0,1306.0,953.0,99.0,176.2,3.62841,4.9392,...,-4.0,-2.697563,41.971107,-87.648315,175.031021,134.804395,10.769385,6.99967,-4.0,89.0


In [47]:
#将X_test1和y_test1合并
xe1=pd.DataFrame(X_test1)
ye1=pd.DataFrame(y_test1)
xe1.columns=dataframe.columns
ye1.columns=['heart_rate_change_value']
xye1=pd.concat((xe1,ye1),axis=1)
xye1.reset_index(drop=True)
xye1.shape #1365,177
xye1.head()
#xye1.to_csv('test_fsMIR.csv')

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
3356,28.0,143.0,110.0,2388.0,2329.0,1595.0,707.0,183.0,5.19322,0.0324,...,25.0,24.751625,41.966806,-87.665936,177.696142,138.496316,11.195287,7.46227,25.0,33.0
1049,77.0,122.0,89.0,635.0,418.0,146.0,14.0,207.8,1.43920,0.1692,...,15.0,15.658145,41.965341,-87.654428,187.561611,152.732323,13.888919,1.83995,15.0,33.0
2364,24.0,168.0,153.0,3395.0,2298.0,102.0,38.0,180.8,9.42123,0.2016,...,16.0,16.221167,41.964517,-87.645495,181.600910,142.907017,14.102744,11.62598,16.0,15.0
1430,11.0,143.0,135.0,1391.0,1348.0,911.0,31.0,199.2,4.98238,0.0000,...,23.0,23.785311,41.965256,-87.647350,182.747063,145.275623,13.423693,5.47680,24.0,8.0
1500,26.0,193.0,109.0,146.0,146.0,146.0,90.0,176.4,0.13136,0.0000,...,10.0,10.437550,41.965223,-87.647369,182.822614,165.372895,15.386804,5.43209,10.0,84.0
